In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report


In [2]:
# Data Augmentation for Training Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Validation and Test Data (No Augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load Training Data
train_data = train_datagen.flow_from_directory(
    'C:\\Users\\ishan\\Downloads\\archive\\chest_xray\\train',  # Replace with your training data path
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='binary'
)

# Load Validation Data
val_data = val_test_datagen.flow_from_directory(
    'C:\\Users\\ishan\\Downloads\\archive\\chest_xray\\val',  # Replace with your validation data path
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='binary'
)

# Load Test Data
test_data = val_test_datagen.flow_from_directory(
    'C:\\Users\\ishan\\Downloads\\archive\\chest_xray\\test',  # Replace with your test data path
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Ensure test data is not shuffled
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))


In [4]:
# Load Pre-trained MobileNetV2
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

# Fine-tune: Freeze the first 100 layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # L2 Regularization
    Dropout(0.4),  # Dropout for regularization
    Dense(1, activation='sigmoid')  # Binary Classification (Normal vs Pneumonia)
])

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Reduce learning rate when a metric stops improving
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

# Stop training early when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [8]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[lr_scheduler, early_stopping]
)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.8636 - loss: 2.0323 - val_accuracy: 0.8125 - val_loss: 2.4577 - learning_rate: 0.0010
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 281s 2s/step - accuracy: 0.9385 - loss: 0.4903 - val_accuracy: 0.6875 - val_loss: 2.7053 - learning_rate: 0.0010
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.9545 - loss: 0.2162 - val_accuracy: 0.9375 - val_loss: 0.1607 - learning_rate: 0.0010
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.9475 - loss: 0.1791 - val_accuracy: 0.8750 - val_loss: 1.3201 - learning_rate: 0.0010
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9494 - loss: 0.1520
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
163/163 ━━━━━━━━━━━━━━━━━━━━ 281s 2s/step - accuracy: 0.9494 - loss: 0.1520 - val_accuracy: 0.5625 - val_loss: 1.6072 - learning_rate: 0.0010
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.9719 - loss: 0.10

In [10]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 574ms/step - accuracy: 0.8989 - loss: 0.5690
Test Accuracy: 91.03%


In [11]:
# Save the model for future use
model.save('pneumonia_detection_model.h5')
